<h1><center>The Battle of the Neighborhoods - Boston</center></h1>

<h1><center>Applied Data Science Capstone Project</center></h1>

<h4>About this Project</h4>
In this project, we will evaluate the Boston area neighborhoods in order to determine the most advantagous areas to live based on a customer's preferences.  

<h4>Discription and background of the problem</h4>
A potential job opening is available in the greater Boston area and a family is considering the best location to search for future homes.  The family has never been to Boston and unfamiliar with the various neighborhoods.  The family has an opportunity to house hunt during a four-day weekend and would like to search neighborhoods that offer the best opportunities as places with live with services and businesses that they frequent.  

<h4>Data and how it will solve the problem</h4>
The Foursquare.com data provides location information on different types of venues by category and name.  The family would like to search neighborhoods based on the proximity to favorite venues.

The family has a number of store preferences and would like to exam the types of venues in Foursquare.com data.  They would like to search specific names of businesses and determine if they can identify the locations with the best range to their preferred venues.

In [1]:
#Prepare the environment
import numpy as np 
import pandas as pd

In [2]:
#Preparing the dependencies
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!pip install lxml

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [3]:
import requests # library to handle requests
import random # library for random number generation
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [4]:
CLIENT_ID = 'PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH' # your Foursquare ID
CLIENT_SECRET = 'RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH
CLIENT_SECRET:RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE


In [5]:
address = '201 Edgewater Drive, Wakefield, MA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

42.5199907 -71.04339433612998


In [6]:
search_query = 'store'
radius = 10000
print(search_query + ' .... OK!')

store .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.5199907,-71.04339433612998&v=20180604&query=store&radius=10000&limit=1000'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e67b298205d001b7c5321'},
 'response': {'venues': [{'id': '5be1033e92e7a9002c62ce84',
    'name': 'TUMI Store - Lynnfield',
    'location': {'address': '600 Market St Spc 687',
     'lat': 42.5164879,
     'lng': -71.0353819,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.5164879,
       'lng': -71.0353819}],
     'distance': 764,
     'postalCode': '01940',
     'cc': 'US',
     'city': 'Lynnfield',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['600 Market St Spc 687',
      'Lynnfield, MA 01940',
      'United States']},
    'categories': [{'id': '52f2ab2ebcbc57f1066b8b29',
      'name': 'Luggage Store',
      'pluralName': 'Luggage Stores',
      'shortName': 'Luggage Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580099580',
    'hasPerk': False},
   {'id': '57e39d8f498ee24a37e658d

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '52f2ab2ebcbc57f1066b8b29', 'name': 'L...",False,5be1033e92e7a9002c62ce84,600 Market St Spc 687,US,Lynnfield,United States,NaN,764,"[600 Market St Spc 687, Lynnfield, MA 01940, U...","[{'label': 'display', 'lat': 42.5164879, 'lng'...",42.516488,-71.035382,01940,MA,TUMI Store - Lynnfield,v-1580099580,NaN
1,"[{'id': '5453de49498eade8af355881', 'name': 'B...",False,57e39d8f498ee24a37e658dc,601 Edgewater Dr,US,Wakefield,United States,NaN,554,"[601 Edgewater Dr, Wakefield, MA 01880, United...","[{'label': 'display', 'lat': 42.52202165068454...",42.522022,-71.049560,01880,MA,XFINITY Store by Comcast,v-1580099580,NaN
2,"[{'id': '4bf58dd8d48988d128951735', 'name': 'G...",False,5241ecb111d2f9f64c050071,NaN,US,Lynnfield,United States,NaN,1089,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.51442, 'lng': ...",42.514420,-71.032470,NaN,MA,The Paper Store,v-1580099580,NaN
3,"[{'id': '4f04afc02fb6e1c99f3db0bc', 'name': 'M...",False,5e0a669353f5420008d40710,225 Market Street,US,Lynnfield,United States,NaN,1206,"[225 Market Street, Lynnfield, MA 01940, Unite...","[{'label': 'display', 'lat': 42.5151073, 'lng'...",42.515107,-71.030266,01940,MA,AT&T Store,v-1580099580,NaN
4,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",False,50dc9fc1e4b067e5d7dbfafe,NaN,US,Wakefield,United States,NaN,1690,"[Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.517091, 'lng':...",42.517091,-71.063617,01880,MA,Medicine Store,v-1580099580,NaN


In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,TUMI Store - Lynnfield,Luggage Store,600 Market St Spc 687,US,Lynnfield,United States,NaN,764,"[600 Market St Spc 687, Lynnfield, MA 01940, U...","[{'label': 'display', 'lat': 42.5164879, 'lng'...",42.516488,-71.035382,01940,MA,5be1033e92e7a9002c62ce84
1,XFINITY Store by Comcast,Business Service,601 Edgewater Dr,US,Wakefield,United States,NaN,554,"[601 Edgewater Dr, Wakefield, MA 01880, United...","[{'label': 'display', 'lat': 42.52202165068454...",42.522022,-71.049560,01880,MA,57e39d8f498ee24a37e658dc
2,The Paper Store,Gift Shop,NaN,US,Lynnfield,United States,NaN,1089,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.51442, 'lng': ...",42.514420,-71.032470,NaN,MA,5241ecb111d2f9f64c050071
3,AT&T Store,Mobile Phone Shop,225 Market Street,US,Lynnfield,United States,NaN,1206,"[225 Market Street, Lynnfield, MA 01940, Unite...","[{'label': 'display', 'lat': 42.5151073, 'lng'...",42.515107,-71.030266,01940,MA,5e0a669353f5420008d40710
4,Medicine Store,Pharmacy,NaN,US,Wakefield,United States,NaN,1690,"[Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.517091, 'lng':...",42.517091,-71.063617,01880,MA,50dc9fc1e4b067e5d7dbfafe
5,C Store,Food & Drink Shop,41 Main St,US,Stoneham,United States,North Street,5282,"[41 Main St (North Street), Stoneham, MA, Unit...","[{'label': 'display', 'lat': 42.499152, 'lng':...",42.499152,-71.101230,NaN,MA,4cdb03cd99026dcb52060a81
6,"Your CBD Store - Reading, MA",Alternative Healer,587 Main St.,US,Reading,United States,NaN,4888,"[587 Main St., Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.5233339, 'lng'...",42.523334,-71.102806,01867,MA,5c639ab6838e5900396bbedd
7,XFINITY Store by Comcast,Business Service,8 Center Vil,US,Lynnfield,United States,NaN,1511,"[8 Center Vil, Lynnfield, MA 01940, United Sta...","[{'label': 'display', 'lat': 42.53220231344306...",42.532202,-71.035334,01940,MA,58e3485284c4ed36c50b090a
8,The UPS Store,Shipping Store,1268C Broadway,US,Saugus,United States,NaN,7410,"[1268C Broadway, Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.45513010433513...",42.455130,-71.023057,01906,MA,56991f3b498e4db57b5dbbf9
9,Sherwin-Williams Paint Store,Hardware Store,900 Broadway,US,Saugus,United States,NaN,5860,"[900 Broadway, Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.46923209991908...",42.469232,-71.024433,01906,MA,50623b4fe4b0ef717cd27bf5


In [11]:
dataframe_filtered.name

0                    TUMI Store - Lynnfield
1                  XFINITY Store by Comcast
2                           The Paper Store
3                                AT&T Store
4                            Medicine Store
5                                   C Store
6              Your CBD Store - Reading, MA
7                  XFINITY Store by Comcast
8                             The UPS Store
9              Sherwin-Williams Paint Store
10                             Amazon Books
11                New Balance Factory Store
12                             Sprint Store
13                              Ryers Store
14                             Sprint Store
15              East Woburn - Package Store
16               Richdale Convenience Store
17                Andrea's Convenient Store
18             Finders Keepers Thrift Store
19                   Jefferys package store
20                            The UPS Store
21                             Sprint Store
22                  The Computer

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new office

# add a red circle marker to represent the new office
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the stores as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [13]:
venues1 = dataframe_filtered
venues1

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,TUMI Store - Lynnfield,Luggage Store,600 Market St Spc 687,US,Lynnfield,United States,NaN,764,"[600 Market St Spc 687, Lynnfield, MA 01940, U...","[{'label': 'display', 'lat': 42.5164879, 'lng'...",42.516488,-71.035382,01940,MA,5be1033e92e7a9002c62ce84
1,XFINITY Store by Comcast,Business Service,601 Edgewater Dr,US,Wakefield,United States,NaN,554,"[601 Edgewater Dr, Wakefield, MA 01880, United...","[{'label': 'display', 'lat': 42.52202165068454...",42.522022,-71.049560,01880,MA,57e39d8f498ee24a37e658dc
2,The Paper Store,Gift Shop,NaN,US,Lynnfield,United States,NaN,1089,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.51442, 'lng': ...",42.514420,-71.032470,NaN,MA,5241ecb111d2f9f64c050071
3,AT&T Store,Mobile Phone Shop,225 Market Street,US,Lynnfield,United States,NaN,1206,"[225 Market Street, Lynnfield, MA 01940, Unite...","[{'label': 'display', 'lat': 42.5151073, 'lng'...",42.515107,-71.030266,01940,MA,5e0a669353f5420008d40710
4,Medicine Store,Pharmacy,NaN,US,Wakefield,United States,NaN,1690,"[Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.517091, 'lng':...",42.517091,-71.063617,01880,MA,50dc9fc1e4b067e5d7dbfafe
5,C Store,Food & Drink Shop,41 Main St,US,Stoneham,United States,North Street,5282,"[41 Main St (North Street), Stoneham, MA, Unit...","[{'label': 'display', 'lat': 42.499152, 'lng':...",42.499152,-71.101230,NaN,MA,4cdb03cd99026dcb52060a81
6,"Your CBD Store - Reading, MA",Alternative Healer,587 Main St.,US,Reading,United States,NaN,4888,"[587 Main St., Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.5233339, 'lng'...",42.523334,-71.102806,01867,MA,5c639ab6838e5900396bbedd
7,XFINITY Store by Comcast,Business Service,8 Center Vil,US,Lynnfield,United States,NaN,1511,"[8 Center Vil, Lynnfield, MA 01940, United Sta...","[{'label': 'display', 'lat': 42.53220231344306...",42.532202,-71.035334,01940,MA,58e3485284c4ed36c50b090a
8,The UPS Store,Shipping Store,1268C Broadway,US,Saugus,United States,NaN,7410,"[1268C Broadway, Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.45513010433513...",42.455130,-71.023057,01906,MA,56991f3b498e4db57b5dbbf9
9,Sherwin-Williams Paint Store,Hardware Store,900 Broadway,US,Saugus,United States,NaN,5860,"[900 Broadway, Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.46923209991908...",42.469232,-71.024433,01906,MA,50623b4fe4b0ef717cd27bf5


In [14]:
venues1_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new office

# add a red circle marker to represent the new office
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues1_map)

# add the stores as blue circle markers
for lat, lng, label in zip(venues1.lat, venues1.lng, venues1.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues1_map)

# display map
venues1_map

In [15]:
#Set the location of the new office
latitude = 42.519
longitude = -71.0433

In [16]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&radius=10000&limit=1000'

In [17]:
import requests
results = requests.get(url).json()
'There are {} around the new office.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around the new office.'

In [18]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '520e153f11d233c87af4422d',
  'name': 'MarketStreet Lynnfield',
  'location': {'address': '600 Market St',
   'crossStreet': 'at Walnut St',
   'lat': 42.51522576983466,
   'lng': -71.03449274955358,
   'labeledLatLngs': [{'label': 'display',
     'lat': 42.51522576983466,
     'lng': -71.03449274955358}],
   'distance': 835,
   'postalCode': '01940',
   'cc': 'US',
   'city': 'Lynnfield',
   'state': 'MA',
   'country': 'United States',
   'formattedAddress': ['600 Market St (at Walnut St)',
    'Lynnfield, MA 01940',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1fd941735',
    'name': 'Shopping Mall',
    'pluralName': 'Shopping Malls',
    'shortName': 'Mall',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 

In [19]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,MarketStreet Lynnfield,Shopping Mall,600 Market St,US,Lynnfield,United States,at Walnut St,835,"[600 Market St (at Walnut St), Lynnfield, MA 0...","[{'label': 'display', 'lat': 42.51522576983466...",42.515226,-71.034493,NaN,01940,MA,520e153f11d233c87af4422d
1,sweetgreen,Salad Place,675 Market St,US,Lynnfield,United States,NaN,676,"[675 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.5167886, 'lng'...",42.516789,-71.035619,NaN,01940,MA,56376bfb498e1a42e86b7b73
2,Whole Foods Market,Grocery Store,427 Walnut Street,US,Lynnfield,United States,NaN,1053,"[427 Walnut Street, Lynnfield, MA 01940, Unite...","[{'label': 'display', 'lat': 42.5144224656762,...",42.514422,-71.032063,NaN,01940,MA,521209e111d244c9730c2449
3,Meletharb Homemade Ice Cream,Ice Cream Shop,393 Lowell St,US,Wakefield,United States,NaN,1680,"[393 Lowell St, Wakefield, MA 01880, United St...","[{'label': 'display', 'lat': 42.51736478257608...",42.517365,-71.063662,NaN,01880,MA,4bafeb51f964a520792b3ce3
4,Starbucks,Coffee Shop,420 Market St,US,Lynnfield,United States,NaN,896,"[420 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.51516539, 'lng...",42.515165,-71.033691,NaN,01940,MA,5216224911d2c0d3d4626e20
5,California Pizza Kitchen,Pizza Place,"1150 Market Street, Lynnfield",US,Lynnfield,United States,NaN,661,"[1150 Market Street, Lynnfield, Lynnfield, MA ...","[{'label': 'display', 'lat': 42.517421, 'lng':...",42.517421,-71.035522,NaN,01940,MA,5754cef9498e8ab8b029702d
6,Apple MarketStreet,Electronics Store,600 Market St,US,Lynnfield,United States,NaN,817,"[600 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.51567799572217...",42.515678,-71.034415,NaN,01940,MA,58637fb5a370b9294234014d
7,J.P. Licks MarketStreet,Ice Cream Shop,800 Market St,US,Lynnfield,United States,NaN,789,"[800 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.51603273747467...",42.516033,-71.034565,NaN,01940,MA,525a0cc711d26b8d7178ef3c
8,Boston Sports Club,Gym / Fitness Center,425 Walnut St,US,Lynnfield,United States,Audubon Rd,738,"[425 Walnut St (Audubon Rd), Lynnfield, MA 019...","[{'label': 'display', 'lat': 42.51476129213743...",42.514761,-71.036369,NaN,01940,MA,4b72b4dff964a520ae842de3
9,Legal C Bar,Seafood Restaurant,220 Market St,US,Lynnfield,United States,NaN,993,"[220 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.5153549476733,...",42.515355,-71.032241,NaN,01940,MA,5250777c8bbdda58c67f00d7


In [20]:
#create a list of stores
shopping = dataframe_filtered
shopping

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,MarketStreet Lynnfield,Shopping Mall,600 Market St,US,Lynnfield,United States,at Walnut St,835,"[600 Market St (at Walnut St), Lynnfield, MA 0...","[{'label': 'display', 'lat': 42.51522576983466...",42.515226,-71.034493,NaN,01940,MA,520e153f11d233c87af4422d
1,sweetgreen,Salad Place,675 Market St,US,Lynnfield,United States,NaN,676,"[675 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.5167886, 'lng'...",42.516789,-71.035619,NaN,01940,MA,56376bfb498e1a42e86b7b73
2,Whole Foods Market,Grocery Store,427 Walnut Street,US,Lynnfield,United States,NaN,1053,"[427 Walnut Street, Lynnfield, MA 01940, Unite...","[{'label': 'display', 'lat': 42.5144224656762,...",42.514422,-71.032063,NaN,01940,MA,521209e111d244c9730c2449
3,Meletharb Homemade Ice Cream,Ice Cream Shop,393 Lowell St,US,Wakefield,United States,NaN,1680,"[393 Lowell St, Wakefield, MA 01880, United St...","[{'label': 'display', 'lat': 42.51736478257608...",42.517365,-71.063662,NaN,01880,MA,4bafeb51f964a520792b3ce3
4,Starbucks,Coffee Shop,420 Market St,US,Lynnfield,United States,NaN,896,"[420 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.51516539, 'lng...",42.515165,-71.033691,NaN,01940,MA,5216224911d2c0d3d4626e20
5,California Pizza Kitchen,Pizza Place,"1150 Market Street, Lynnfield",US,Lynnfield,United States,NaN,661,"[1150 Market Street, Lynnfield, Lynnfield, MA ...","[{'label': 'display', 'lat': 42.517421, 'lng':...",42.517421,-71.035522,NaN,01940,MA,5754cef9498e8ab8b029702d
6,Apple MarketStreet,Electronics Store,600 Market St,US,Lynnfield,United States,NaN,817,"[600 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.51567799572217...",42.515678,-71.034415,NaN,01940,MA,58637fb5a370b9294234014d
7,J.P. Licks MarketStreet,Ice Cream Shop,800 Market St,US,Lynnfield,United States,NaN,789,"[800 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.51603273747467...",42.516033,-71.034565,NaN,01940,MA,525a0cc711d26b8d7178ef3c
8,Boston Sports Club,Gym / Fitness Center,425 Walnut St,US,Lynnfield,United States,Audubon Rd,738,"[425 Walnut St (Audubon Rd), Lynnfield, MA 019...","[{'label': 'display', 'lat': 42.51476129213743...",42.514761,-71.036369,NaN,01940,MA,4b72b4dff964a520ae842de3
9,Legal C Bar,Seafood Restaurant,220 Market St,US,Lynnfield,United States,NaN,993,"[220 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.5153549476733,...",42.515355,-71.032241,NaN,01940,MA,5250777c8bbdda58c67f00d7


<h4>The family wants to explore the area around the new office building</h4>
The map provides the family with various venues around the new office building.

In [21]:
shopping_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new office


# add new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(shopping_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(shopping.lat, shopping.lng, shopping.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(shopping_map)

# display map
shopping_map

<h4>The family wants to see where all the malls near the new office.</h4>
The map provides the family with the malls near the new office building.

In [22]:
search_query = 'Shopping Mall'
radius = 10000
print(search_query + ' .... OK!')

Shopping Mall .... OK!


In [23]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=Shopping Mall&radius=10000&limit=1000'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e67dc963d29001b7738b5'},
 'response': {'venues': [{'id': '472b28e9f964a520f54b1fe3',
    'name': 'Northshore Mall',
    'location': {'address': '210 Andover St',
     'crossStreet': 'at Prospect St',
     'lat': 42.540465,
     'lng': -70.9427116,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.540465,
       'lng': -70.9427116}],
     'distance': 8590,
     'postalCode': '01960',
     'cc': 'US',
     'city': 'Peabody',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['210 Andover St (at Prospect St)',
      'Peabody, MA 01960',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fd941735',
      'name': 'Shopping Mall',
      'pluralName': 'Shopping Malls',
      'shortName': 'Mall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580099712',
    'hasPerk': False},
   {'id': '4b0f0e9

In [25]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,472b28e9f964a520f54b1fe3,210 Andover St,US,Peabody,United States,at Prospect St,8590,"[210 Andover St (at Prospect St), Peabody, MA ...","[{'label': 'display', 'lat': 42.540465, 'lng':...",42.540465,-70.942712,01960,MA,Northshore Mall,v-1580099712,NaN
1,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b0f0e95f964a520a85e23e3,100 Independence Way,US,Danvers,United States,at Liberty Tree Mall,9193,"[100 Independence Way (at Liberty Tree Mall), ...","[{'label': 'display', 'lat': 42.55266502119669...",42.552665,-70.940950,01923,MA,AMC Loews Liberty Tree Mall 20,v-1580099712,NaN
2,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bbcfabb8ec3d13ab9811b28,296 Mishawum Rd,US,Woburn,United States,NaN,7498,"[296 Mishawum Rd, Woburn, MA 01801, United Sta...","[{'label': 'display', 'lat': 42.50488838716877...",42.504888,-71.132658,01801,MA,Woburn Mall,v-1580099712,NaN
3,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,472b2475f964a520f44b1fe3,100 Independence Way,US,Danvers,United States,at Commonwealth Ave,9249,"[100 Independence Way (at Commonwealth Ave), D...","[{'label': 'display', 'lat': 42.5521762, 'lng'...",42.552176,-70.939918,01923,MA,Liberty Tree Mall,v-1580099712,NaN
4,"[{'id': '4bf58dd8d48988d108951735', 'name': 'W...",NaN,NaN,NaN,NaN,NaN,NaN,False,50c935cbe4b07110f3609f8e,NaN,US,Saugus,United States,NaN,3890,"[Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.48871612548828...",42.488716,-71.019630,01906,MA,square one mall-debs,v-1580099712,NaN


In [26]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Northshore Mall,Shopping Mall,210 Andover St,US,Peabody,United States,at Prospect St,8590,"[210 Andover St (at Prospect St), Peabody, MA ...","[{'label': 'display', 'lat': 42.540465, 'lng':...",42.540465,-70.942712,01960,MA,472b28e9f964a520f54b1fe3
1,AMC Loews Liberty Tree Mall 20,Movie Theater,100 Independence Way,US,Danvers,United States,at Liberty Tree Mall,9193,"[100 Independence Way (at Liberty Tree Mall), ...","[{'label': 'display', 'lat': 42.55266502119669...",42.552665,-70.940950,01923,MA,4b0f0e95f964a520a85e23e3
2,Woburn Mall,Shopping Mall,296 Mishawum Rd,US,Woburn,United States,NaN,7498,"[296 Mishawum Rd, Woburn, MA 01801, United Sta...","[{'label': 'display', 'lat': 42.50488838716877...",42.504888,-71.132658,01801,MA,4bbcfabb8ec3d13ab9811b28
3,Liberty Tree Mall,Shopping Mall,100 Independence Way,US,Danvers,United States,at Commonwealth Ave,9249,"[100 Independence Way (at Commonwealth Ave), D...","[{'label': 'display', 'lat': 42.5521762, 'lng'...",42.552176,-70.939918,01923,MA,472b2475f964a520f44b1fe3
4,square one mall-debs,Women's Store,NaN,US,Saugus,United States,NaN,3890,"[Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.48871612548828...",42.488716,-71.019630,01906,MA,50c935cbe4b07110f3609f8e
5,Burtons North Shore Mall,American Restaurant,NaN,US,Danvers,United States,NaN,4439,"[Danvers, MA, United States]","[{'label': 'display', 'lat': 42.5208750208098,...",42.520875,-70.989250,NaN,MA,4dbc51a11e72b351caafa708
6,Northshore Mall rd,Road,NaN,US,Peabody,United States,NaN,8541,"[Peabody, MA 01960, United States]","[{'label': 'display', 'lat': 42.53777241638895...",42.537772,-70.942341,01960,MA,507cdb5ae4b08ed44a906f58
7,northshore mall parking lot,Parking,NaN,US,Peabody,United States,NaN,6001,"[Peabody, MA 01960, United States]","[{'label': 'display', 'lat': 42.5425265, 'lng'...",42.542527,-70.977468,01960,MA,500dae08e4b03bf190df8d68
8,Redstone Shopping Center,Shopping Mall,65 Main St,US,Stoneham,United States,NaN,5449,"[65 Main St, Stoneham, MA 02180, United States]","[{'label': 'display', 'lat': 42.49509895626455...",42.495099,-71.101248,02180,MA,4c5ff1e5de6920a1a1b79464
9,Mall Play Space,Shopping Mall,NaN,US,NaN,United States,NaN,5996,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.54435729980469...",42.544357,-70.978806,NaN,Massachusetts,4e3d6245b61cb577be75f656


In [27]:
#create a list of shopping malls
malls = dataframe_filtered
malls

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Northshore Mall,Shopping Mall,210 Andover St,US,Peabody,United States,at Prospect St,8590,"[210 Andover St (at Prospect St), Peabody, MA ...","[{'label': 'display', 'lat': 42.540465, 'lng':...",42.540465,-70.942712,01960,MA,472b28e9f964a520f54b1fe3
1,AMC Loews Liberty Tree Mall 20,Movie Theater,100 Independence Way,US,Danvers,United States,at Liberty Tree Mall,9193,"[100 Independence Way (at Liberty Tree Mall), ...","[{'label': 'display', 'lat': 42.55266502119669...",42.552665,-70.940950,01923,MA,4b0f0e95f964a520a85e23e3
2,Woburn Mall,Shopping Mall,296 Mishawum Rd,US,Woburn,United States,NaN,7498,"[296 Mishawum Rd, Woburn, MA 01801, United Sta...","[{'label': 'display', 'lat': 42.50488838716877...",42.504888,-71.132658,01801,MA,4bbcfabb8ec3d13ab9811b28
3,Liberty Tree Mall,Shopping Mall,100 Independence Way,US,Danvers,United States,at Commonwealth Ave,9249,"[100 Independence Way (at Commonwealth Ave), D...","[{'label': 'display', 'lat': 42.5521762, 'lng'...",42.552176,-70.939918,01923,MA,472b2475f964a520f44b1fe3
4,square one mall-debs,Women's Store,NaN,US,Saugus,United States,NaN,3890,"[Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.48871612548828...",42.488716,-71.019630,01906,MA,50c935cbe4b07110f3609f8e
5,Burtons North Shore Mall,American Restaurant,NaN,US,Danvers,United States,NaN,4439,"[Danvers, MA, United States]","[{'label': 'display', 'lat': 42.5208750208098,...",42.520875,-70.989250,NaN,MA,4dbc51a11e72b351caafa708
6,Northshore Mall rd,Road,NaN,US,Peabody,United States,NaN,8541,"[Peabody, MA 01960, United States]","[{'label': 'display', 'lat': 42.53777241638895...",42.537772,-70.942341,01960,MA,507cdb5ae4b08ed44a906f58
7,northshore mall parking lot,Parking,NaN,US,Peabody,United States,NaN,6001,"[Peabody, MA 01960, United States]","[{'label': 'display', 'lat': 42.5425265, 'lng'...",42.542527,-70.977468,01960,MA,500dae08e4b03bf190df8d68
8,Redstone Shopping Center,Shopping Mall,65 Main St,US,Stoneham,United States,NaN,5449,"[65 Main St, Stoneham, MA 02180, United States]","[{'label': 'display', 'lat': 42.49509895626455...",42.495099,-71.101248,02180,MA,4c5ff1e5de6920a1a1b79464
9,Mall Play Space,Shopping Mall,NaN,US,NaN,United States,NaN,5996,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.54435729980469...",42.544357,-70.978806,NaN,Massachusetts,4e3d6245b61cb577be75f656


In [28]:
options = ['Shopping Mall'] 
  
# selecting rows based on condition 
malls_df = malls[dataframe_filtered['categories'].isin(options)] 
  
malls_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Northshore Mall,Shopping Mall,210 Andover St,US,Peabody,United States,at Prospect St,8590,"[210 Andover St (at Prospect St), Peabody, MA ...","[{'label': 'display', 'lat': 42.540465, 'lng':...",42.540465,-70.942712,01960,MA,472b28e9f964a520f54b1fe3
2,Woburn Mall,Shopping Mall,296 Mishawum Rd,US,Woburn,United States,NaN,7498,"[296 Mishawum Rd, Woburn, MA 01801, United Sta...","[{'label': 'display', 'lat': 42.50488838716877...",42.504888,-71.132658,01801,MA,4bbcfabb8ec3d13ab9811b28
3,Liberty Tree Mall,Shopping Mall,100 Independence Way,US,Danvers,United States,at Commonwealth Ave,9249,"[100 Independence Way (at Commonwealth Ave), D...","[{'label': 'display', 'lat': 42.5521762, 'lng'...",42.552176,-70.939918,01923,MA,472b2475f964a520f44b1fe3
8,Redstone Shopping Center,Shopping Mall,65 Main St,US,Stoneham,United States,NaN,5449,"[65 Main St, Stoneham, MA 02180, United States]","[{'label': 'display', 'lat': 42.49509895626455...",42.495099,-71.101248,02180,MA,4c5ff1e5de6920a1a1b79464
9,Mall Play Space,Shopping Mall,NaN,US,NaN,United States,NaN,5996,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.54435729980469...",42.544357,-70.978806,NaN,Massachusetts,4e3d6245b61cb577be75f656
14,Square One Mall,Shopping Mall,1201 Broadway,US,Saugus,United States,at Essex St,6604,"[1201 Broadway (at Essex St), Saugus, MA 01906...","[{'label': 'display', 'lat': 42.46083411679223...",42.460834,-71.027447,01906,MA,4a56e4b2f964a520e1b51fe3
18,Square One Mall,Shopping Mall,NaN,US,NaN,United States,NaN,9727,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.461032, 'lng':...",42.461032,-70.954627,NaN,Massachusetts,4d8e4f4fd4ec8cfa9b627089


In [29]:
malls_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new office


# add the new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(malls_map)


# add malls to the map as blue circle markers
for lat, lng, label in zip(malls_df.lat, malls_df.lng, malls_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='black',
        fill_color='black',
        fill_opacity=0.6
        ).add_to(malls_map)

# display map
malls_map

<h4>The family likes to shop at organic grocery stores</h4>
The map provides the family with the locations of organic grocery stores near the new office.

In [30]:
search_query = 'Organic Grocery'
radius = 100000
print(search_query + ' .... OK!')

Organic Grocery .... OK!


In [31]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=Organic Grocery&radius=100000&limit=1000'

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e690e5fb726001bd433a7'},
 'response': {'venues': [{'id': '4e5f955062e19bc92adb6626',
    'name': 'Organic Café',
    'location': {'lat': 42.5680446261517,
     'lng': -70.97508661671253,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.5680446261517,
       'lng': -70.97508661671253}],
     'distance': 7817,
     'cc': 'US',
     'city': 'Danvers',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Danvers, MA', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1d3941735',
      'name': 'Vegetarian / Vegan Restaurant',
      'pluralName': 'Vegetarian / Vegan Restaurants',
      'shortName': 'Vegetarian / Vegan',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580099771',
    'hasPerk': False},
   {'id': '5348882f498e2eb1c626c106',
    'name': 'Sakura Organic',
    'location': {'lat': 42.

In [33]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",NaN,NaN,NaN,NaN,NaN,NaN,False,4e5f955062e19bc92adb6626,NaN,US,Danvers,United States,NaN,7817,"[Danvers, MA, United States]","[{'label': 'display', 'lat': 42.5680446261517,...",42.568045,-70.975087,NaN,MA,Organic Café,v-1580099771
1,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",287925,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/sakura-orga...,False,5348882f498e2eb1c626c106,NaN,US,Wakefield,United States,NaN,2823,"[Wakefield, MA, United States]","[{'label': 'display', 'lat': 42.50333265019249...",42.503333,-71.070357,NaN,MA,Sakura Organic,v-1580099771
2,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,5a58021f270ee73507258cf4,321 Main St #1935,US,Wakefield,United States,NaN,2837,"[321 Main St #1935, Wakefield, MA 01880, Unite...","[{'label': 'display', 'lat': 42.5048772, 'lng'...",42.504877,-71.072088,01880,MA,Velvet Green Organic Lawn Care,v-1580099771
3,"[{'id': '4f04aa0c2fb6e1c99f3db0b8', 'name': 'N...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3e7b8ae4b090fd1727db60,NaN,US,Arlington,United States,NaN,15514,"[Arlington, MA 02476, United States]","[{'label': 'display', 'lat': 42.424462, 'lng':...",42.424462,-71.182130,02476,MA,Organic Nail Lounge,v-1580099771
4,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,51af674f498e5953afa1712e,267 Rantoul St,US,Beverly,United States,at Federal St,13588,"[267 Rantoul St (at Federal St), Beverly, MA 0...","[{'label': 'display', 'lat': 42.550255, 'lng':...",42.550255,-70.883170,01915,MA,The Organic Rainbow,v-1580099771


In [34]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Organic Café,Vegetarian / Vegan Restaurant,NaN,US,Danvers,United States,NaN,7817,"[Danvers, MA, United States]","[{'label': 'display', 'lat': 42.5680446261517,...",42.568045,-70.975087,NaN,MA,4e5f955062e19bc92adb6626
1,Sakura Organic,Sushi Restaurant,NaN,US,Wakefield,United States,NaN,2823,"[Wakefield, MA, United States]","[{'label': 'display', 'lat': 42.50333265019249...",42.503333,-71.070357,NaN,MA,5348882f498e2eb1c626c106
2,Velvet Green Organic Lawn Care,Construction & Landscaping,321 Main St #1935,US,Wakefield,United States,NaN,2837,"[321 Main St #1935, Wakefield, MA 01880, Unite...","[{'label': 'display', 'lat': 42.5048772, 'lng'...",42.504877,-71.072088,01880,MA,5a58021f270ee73507258cf4
3,Organic Nail Lounge,Nail Salon,NaN,US,Arlington,United States,NaN,15514,"[Arlington, MA 02476, United States]","[{'label': 'display', 'lat': 42.424462, 'lng':...",42.424462,-71.182130,02476,MA,4f3e7b8ae4b090fd1727db60
4,The Organic Rainbow,Health Food Store,267 Rantoul St,US,Beverly,United States,at Federal St,13588,"[267 Rantoul St (at Federal St), Beverly, MA 0...","[{'label': 'display', 'lat': 42.550255, 'lng':...",42.550255,-70.883170,01915,MA,51af674f498e5953afa1712e
5,Organic Restaurants,Restaurant,175 William F McClellan Hwy,US,Boston,United States,NaN,14978,"[175 William F McClellan Hwy, Boston, MA 02128...","[{'label': 'display', 'lat': 42.3858847, 'lng'...",42.385885,-71.016735,02128,MA,560afcd3498eec335e720ad6
6,Organic Garden Vegetarian,Vegetarian / Vegan Restaurant,294 Cabot St,US,Beverly,United States,NaN,13974,"[294 Cabot St, Beverly, MA 01915, United States]","[{'label': 'display', 'lat': 42.55183145169560...",42.551831,-70.878866,01915,MA,4b6afb49f964a52067ea2be3
7,Sakura Organic,None,397 Main St,US,Wakefield,United States,NaN,8885,"[397 Main St, Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.44170768849803...",42.441708,-71.070341,01880,MA,52ca083f11d217ba4d1f3aaf
8,organic nail and spa,Nail Salon,NaN,US,Lexington,United States,NaN,16983,"[Lexington, MA, United States]","[{'label': 'display', 'lat': 42.447529, 'lng':...",42.447529,-71.226074,NaN,MA,55abc1fa498e73eddd7d04a2
9,Organic Bath Company,Salon / Barbershop,283 Franklin St,US,Boston,United States,NaN,18063,"[283 Franklin St, Boston, MA 02110, United Sta...","[{'label': 'display', 'lat': 42.35689, 'lng': ...",42.356890,-71.053050,02110,MA,55102710498e3f22bf692524


In [35]:
#create a list of grocery stores
grocery = dataframe_filtered
grocery

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Organic Café,Vegetarian / Vegan Restaurant,NaN,US,Danvers,United States,NaN,7817,"[Danvers, MA, United States]","[{'label': 'display', 'lat': 42.5680446261517,...",42.568045,-70.975087,NaN,MA,4e5f955062e19bc92adb6626
1,Sakura Organic,Sushi Restaurant,NaN,US,Wakefield,United States,NaN,2823,"[Wakefield, MA, United States]","[{'label': 'display', 'lat': 42.50333265019249...",42.503333,-71.070357,NaN,MA,5348882f498e2eb1c626c106
2,Velvet Green Organic Lawn Care,Construction & Landscaping,321 Main St #1935,US,Wakefield,United States,NaN,2837,"[321 Main St #1935, Wakefield, MA 01880, Unite...","[{'label': 'display', 'lat': 42.5048772, 'lng'...",42.504877,-71.072088,01880,MA,5a58021f270ee73507258cf4
3,Organic Nail Lounge,Nail Salon,NaN,US,Arlington,United States,NaN,15514,"[Arlington, MA 02476, United States]","[{'label': 'display', 'lat': 42.424462, 'lng':...",42.424462,-71.182130,02476,MA,4f3e7b8ae4b090fd1727db60
4,The Organic Rainbow,Health Food Store,267 Rantoul St,US,Beverly,United States,at Federal St,13588,"[267 Rantoul St (at Federal St), Beverly, MA 0...","[{'label': 'display', 'lat': 42.550255, 'lng':...",42.550255,-70.883170,01915,MA,51af674f498e5953afa1712e
5,Organic Restaurants,Restaurant,175 William F McClellan Hwy,US,Boston,United States,NaN,14978,"[175 William F McClellan Hwy, Boston, MA 02128...","[{'label': 'display', 'lat': 42.3858847, 'lng'...",42.385885,-71.016735,02128,MA,560afcd3498eec335e720ad6
6,Organic Garden Vegetarian,Vegetarian / Vegan Restaurant,294 Cabot St,US,Beverly,United States,NaN,13974,"[294 Cabot St, Beverly, MA 01915, United States]","[{'label': 'display', 'lat': 42.55183145169560...",42.551831,-70.878866,01915,MA,4b6afb49f964a52067ea2be3
7,Sakura Organic,None,397 Main St,US,Wakefield,United States,NaN,8885,"[397 Main St, Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.44170768849803...",42.441708,-71.070341,01880,MA,52ca083f11d217ba4d1f3aaf
8,organic nail and spa,Nail Salon,NaN,US,Lexington,United States,NaN,16983,"[Lexington, MA, United States]","[{'label': 'display', 'lat': 42.447529, 'lng':...",42.447529,-71.226074,NaN,MA,55abc1fa498e73eddd7d04a2
9,Organic Bath Company,Salon / Barbershop,283 Franklin St,US,Boston,United States,NaN,18063,"[283 Franklin St, Boston, MA 02110, United Sta...","[{'label': 'display', 'lat': 42.35689, 'lng': ...",42.356890,-71.053050,02110,MA,55102710498e3f22bf692524


In [36]:
options = ['Grocery Store'] 
  
# selecting rows based on condition 
grocery_df = grocery[dataframe_filtered['categories'].isin(options)] 
  
grocery_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
32,Xpect Grocery & Deep Discount Stores,Grocery Store,436 Broadway,US,Methuen,United States,NaN,27035,"[436 Broadway, Methuen, MA 01844, United States]","[{'label': 'display', 'lat': 42.735078, 'lng':...",42.735078,-71.193990,01844,MA,5239d48911d2da4becb86914
37,Walmart Grocery Pickup & Delivery,Grocery Store,326 N Broadway,US,Salem,United States,NaN,35413,"[326 N Broadway, Salem, NH 03079, United States]","[{'label': 'display', 'lat': 42.8016584011, 'l...",42.801658,-71.241800,03079,NH,5c4b7ee4f129b5002555683d
44,Walmart Grocery Pickup & Delivery,Grocery Store,550 Providence Hwy,US,Walpole,United States,NaN,43645,"[550 Providence Hwy, Walpole, MA 02081, United...","[{'label': 'display', 'lat': 42.1480617476224,...",42.148062,-71.215112,02081,MA,5c4b807fb3c961002c3c1524
45,Walmart Grocery Pickup & Delivery,Grocery Store,30 Memorial Dr,US,Avon,United States,NaN,44265,"[30 Memorial Dr, Avon, MA 02322, United States]","[{'label': 'display', 'lat': 42.12143, 'lng': ...",42.121430,-71.032940,02322,MA,5aab07aeee71203bce3f0806
49,Walmart Grocery Pickup and Delivery,Grocery Store,85 State Route 101A,US,Amherst,United States,NaN,54711,"[85 State Route 101A, Amherst, NH 03031, Unite...","[{'label': 'display', 'lat': 42.81145, 'lng': ...",42.811450,-71.580490,03031,NH,5aab07f3ee71201c3682c272


In [37]:
grocery_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around New Office


# add New Office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(grocery_map)


# add grocery to the map as blue circle markers
for lat, lng, label in zip(grocery_df.lat, grocery_df.lng, grocery_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(grocery_map)

# display map
grocery_map

<h4>The family likes to go to a gym three times per week.</h4>
The map provides the family with the locations of gyms near the new office.

In [38]:
search_query = 'Gym'
radius = 10000
print(search_query + ' .... OK!')

Gym .... OK!


In [39]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=Gym&radius=10000&limit=1000'

In [40]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e6910b57e88001b645625'},
 'response': {'venues': [{'id': '4e2a06afd4c0e5c89ab4ea4e',
    'name': 'Gym',
    'location': {'address': 'Best Western New Englander',
     'crossStreet': '1 Rainin Road',
     'lat': 42.48803964960014,
     'lng': -71.11794289833213,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.48803964960014,
       'lng': -71.11794289833213}],
     'distance': 7028,
     'postalCode': '01801',
     'cc': 'US',
     'city': 'Woburn',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Best Western New Englander (1 Rainin Road)',
      'Woburn, MA 01801',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580099820',
    'hasPerk': False},

In [41]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4e2a06afd4c0e5c89ab4ea4e,Best Western New Englander,US,Woburn,United States,1 Rainin Road,7028,"[Best Western New Englander (1 Rainin Road), W...","[{'label': 'display', 'lat': 42.48803964960014...",42.488040,-71.117943,01801,MA,Gym,v-1580099820
1,"[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...",False,57166a12498e33ed2245020a,1 General Way,US,Reading,United States,NaN,4702,"[1 General Way, Reading, MA 01876, United States]","[{'label': 'display', 'lat': 42.51933951918963...",42.519340,-71.100610,01876,MA,Gold's Gym,v-1580099820
2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4d436d122e326ea8778dd9a6,100 Lowell Rd,US,North Reading,United States,NaN,9789,"[100 Lowell Rd, North Reading, MA 01864, Unite...","[{'label': 'display', 'lat': 42.58407120701979...",42.584071,-71.123601,01864,MA,Gym at Edgewood,v-1580099820
3,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,57080ac6cd100e4615608394,NaN,US,Lynnfield,United States,NaN,2812,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.53879419905821...",42.538794,-71.064613,NaN,MA,Mellow Yellow Gym,v-1580099820
4,"[{'id': '4bf58dd8d48988d1a9941735', 'name': 'C...",False,4ba4ff38f964a5201ece38e3,29 Andover St,US,Danvers,United States,NaN,7958,"[29 Andover St, Danvers, MA 01923, United States]","[{'label': 'display', 'lat': 42.55246781136085...",42.552468,-70.957562,01923,MA,The Little Gym of Danvers,v-1580099820


In [42]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gym,Gym,Best Western New Englander,US,Woburn,United States,1 Rainin Road,7028,"[Best Western New Englander (1 Rainin Road), W...","[{'label': 'display', 'lat': 42.48803964960014...",42.488040,-71.117943,01801,MA,4e2a06afd4c0e5c89ab4ea4e
1,Gold's Gym,Athletics & Sports,1 General Way,US,Reading,United States,NaN,4702,"[1 General Way, Reading, MA 01876, United States]","[{'label': 'display', 'lat': 42.51933951918963...",42.519340,-71.100610,01876,MA,57166a12498e33ed2245020a
2,Gym at Edgewood,Gym,100 Lowell Rd,US,North Reading,United States,NaN,9789,"[100 Lowell Rd, North Reading, MA 01864, Unite...","[{'label': 'display', 'lat': 42.58407120701979...",42.584071,-71.123601,01864,MA,4d436d122e326ea8778dd9a6
3,Mellow Yellow Gym,Gym,NaN,US,Lynnfield,United States,NaN,2812,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.53879419905821...",42.538794,-71.064613,NaN,MA,57080ac6cd100e4615608394
4,The Little Gym of Danvers,College Rec Center,29 Andover St,US,Danvers,United States,NaN,7958,"[29 Andover St, Danvers, MA 01923, United States]","[{'label': 'display', 'lat': 42.55246781136085...",42.552468,-70.957562,01923,MA,4ba4ff38f964a5201ece38e3
5,fuji gym,Gym / Fitness Center,19 New Salem St,US,Wakefield,United States,NaN,2336,"[19 New Salem St, Wakefield, MA 01880, United ...","[{'label': 'display', 'lat': 42.505176, 'lng':...",42.505176,-71.064734,01880,MA,5aa9217d92e7a94231c2e03b
6,gym source at Jordan's Furniture,Sporting Goods Shop,NaN,US,Reading,United States,NaN,3911,"[Reading, MA, United States]","[{'label': 'display', 'lat': 42.520514, 'lng':...",42.520514,-71.090935,NaN,MA,574b615b498eeaadc7fb23a7
7,gym,Track,NaN,US,NaN,United States,NaN,7785,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.57612187469009...",42.576122,-70.988524,NaN,Massachusetts,4f55a5c3e4b072b51d782d35
8,Public Safety Bldg Gym,Gym / Fitness Center,1 Union St,US,Wakefield,United States,princess st,2697,"[1 Union St (princess st), Wakefield, MA 01880...","[{'label': 'display', 'lat': 42.504199, 'lng':...",42.504199,-71.069333,01880,MA,4db98d81fa8c978590d28226
9,Gymboree,Kids Store,NaN,US,Lynnfield,United States,NaN,902,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.515327, 'lng':...",42.515327,-71.033501,NaN,MA,523e154211d26f23616fe504


In [43]:
#create a list of gyms
gym = dataframe_filtered
gym

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gym,Gym,Best Western New Englander,US,Woburn,United States,1 Rainin Road,7028,"[Best Western New Englander (1 Rainin Road), W...","[{'label': 'display', 'lat': 42.48803964960014...",42.488040,-71.117943,01801,MA,4e2a06afd4c0e5c89ab4ea4e
1,Gold's Gym,Athletics & Sports,1 General Way,US,Reading,United States,NaN,4702,"[1 General Way, Reading, MA 01876, United States]","[{'label': 'display', 'lat': 42.51933951918963...",42.519340,-71.100610,01876,MA,57166a12498e33ed2245020a
2,Gym at Edgewood,Gym,100 Lowell Rd,US,North Reading,United States,NaN,9789,"[100 Lowell Rd, North Reading, MA 01864, Unite...","[{'label': 'display', 'lat': 42.58407120701979...",42.584071,-71.123601,01864,MA,4d436d122e326ea8778dd9a6
3,Mellow Yellow Gym,Gym,NaN,US,Lynnfield,United States,NaN,2812,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.53879419905821...",42.538794,-71.064613,NaN,MA,57080ac6cd100e4615608394
4,The Little Gym of Danvers,College Rec Center,29 Andover St,US,Danvers,United States,NaN,7958,"[29 Andover St, Danvers, MA 01923, United States]","[{'label': 'display', 'lat': 42.55246781136085...",42.552468,-70.957562,01923,MA,4ba4ff38f964a5201ece38e3
5,fuji gym,Gym / Fitness Center,19 New Salem St,US,Wakefield,United States,NaN,2336,"[19 New Salem St, Wakefield, MA 01880, United ...","[{'label': 'display', 'lat': 42.505176, 'lng':...",42.505176,-71.064734,01880,MA,5aa9217d92e7a94231c2e03b
6,gym source at Jordan's Furniture,Sporting Goods Shop,NaN,US,Reading,United States,NaN,3911,"[Reading, MA, United States]","[{'label': 'display', 'lat': 42.520514, 'lng':...",42.520514,-71.090935,NaN,MA,574b615b498eeaadc7fb23a7
7,gym,Track,NaN,US,NaN,United States,NaN,7785,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.57612187469009...",42.576122,-70.988524,NaN,Massachusetts,4f55a5c3e4b072b51d782d35
8,Public Safety Bldg Gym,Gym / Fitness Center,1 Union St,US,Wakefield,United States,princess st,2697,"[1 Union St (princess st), Wakefield, MA 01880...","[{'label': 'display', 'lat': 42.504199, 'lng':...",42.504199,-71.069333,01880,MA,4db98d81fa8c978590d28226
9,Gymboree,Kids Store,NaN,US,Lynnfield,United States,NaN,902,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.515327, 'lng':...",42.515327,-71.033501,NaN,MA,523e154211d26f23616fe504


In [44]:
options = ['Gym', 'Gym / Fitness Center'] 
  
# selecting rows based on condition 
gym_df = gym[dataframe_filtered['categories'].isin(options)] 
  
gym_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gym,Gym,Best Western New Englander,US,Woburn,United States,1 Rainin Road,7028,"[Best Western New Englander (1 Rainin Road), W...","[{'label': 'display', 'lat': 42.48803964960014...",42.488040,-71.117943,01801,MA,4e2a06afd4c0e5c89ab4ea4e
2,Gym at Edgewood,Gym,100 Lowell Rd,US,North Reading,United States,NaN,9789,"[100 Lowell Rd, North Reading, MA 01864, Unite...","[{'label': 'display', 'lat': 42.58407120701979...",42.584071,-71.123601,01864,MA,4d436d122e326ea8778dd9a6
3,Mellow Yellow Gym,Gym,NaN,US,Lynnfield,United States,NaN,2812,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.53879419905821...",42.538794,-71.064613,NaN,MA,57080ac6cd100e4615608394
5,fuji gym,Gym / Fitness Center,19 New Salem St,US,Wakefield,United States,NaN,2336,"[19 New Salem St, Wakefield, MA 01880, United ...","[{'label': 'display', 'lat': 42.505176, 'lng':...",42.505176,-71.064734,01880,MA,5aa9217d92e7a94231c2e03b
8,Public Safety Bldg Gym,Gym / Fitness Center,1 Union St,US,Wakefield,United States,princess st,2697,"[1 Union St (princess st), Wakefield, MA 01880...","[{'label': 'display', 'lat': 42.504199, 'lng':...",42.504199,-71.069333,01880,MA,4db98d81fa8c978590d28226
10,Quietman Sports Gym,Gym,NaN,US,Lynnfield,United States,NaN,3311,"[Lynnfield, MA 01940, United States]","[{'label': 'display', 'lat': 42.510996, 'lng':...",42.510996,-71.004426,01940,MA,4f914c03e4b0ff5a5163b3b3
12,Mike's Gym,Gym / Fitness Center,10 Broadway St,US,Wakefield,United States,NaN,3291,"[10 Broadway St, Wakefield, MA 01880, United S...","[{'label': 'display', 'lat': 42.49717334246895...",42.497173,-71.070364,01880,MA,4e5e3f531838f725527300ab
13,Gym,Gym,NaN,US,NaN,United States,NaN,11846,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.4212517753186,...",42.421252,-70.986272,NaN,Massachusetts,4e89f2b846909b71217e6407
15,Gym at Alterra,Gym,11 Overlook Ridge Dr,US,Revere,United States,NaN,9131,"[11 Overlook Ridge Dr, Revere, MA 02151, Unite...","[{'label': 'display', 'lat': 42.43885716796143...",42.438857,-71.019586,02151,MA,4e3073f22271352f543f3f4f
16,Powerfit Gym,Gym,NaN,US,Peabody,United States,NaN,5814,"[Peabody, MA 01960, United States]","[{'label': 'display', 'lat': 42.546903, 'lng':...",42.546903,-70.983381,01960,MA,50f16751e4b036c5c6ad8742


In [45]:
gym_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(gym_map)


# add gyms to the map as purple circle markers
for lat, lng, label in zip(gym_df.lat, gym_df.lng, gym_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='purple',
        fill_color='purple',
        fill_opacity=0.6
        ).add_to(gym_map)

# display map
gym_map

In [46]:
search_query = 'Train'
radius = 10000
print(search_query + ' .... OK!')

Train .... OK!


In [47]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=Train&radius=10000&limit=1000'

In [48]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e6878963d29001b7910da'},
 'response': {'venues': [{'id': '5832d1d0dfa6ff0f03038b5a',
    'name': 'MBTA Commuter Rail Train #200',
    'location': {'lat': 42.521294,
     'lng': -71.10756,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.521294,
       'lng': -71.10756}],
     'distance': 5278,
     'postalCode': '01867',
     'cc': 'US',
     'city': 'Reading',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Reading, MA 01867', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fd931735',
      'name': 'Metro Station',
      'pluralName': 'Metro Stations',
      'shortName': 'Metro',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/subway_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580099868',
    'hasPerk': False},
   {'id': '50226d7be4b079960ffad394',
    'name': 'MBTA Commuter Rail Train #214',
    'location': {'address': '12:41pm Hav

In [49]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,5832d1d0dfa6ff0f03038b5a,NaN,US,Reading,United States,NaN,5278,"[Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.521294, 'lng':...",42.521294,-71.107560,01867,MA,MBTA Commuter Rail Train #200,v-1580099868,NaN
1,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,50226d7be4b079960ffad394,12:41pm Haverhill to Boston,US,NaN,United States,NaN,2051,"[12:41pm Haverhill to Boston, MA 02148, United...","[{'label': 'display', 'lat': 42.52117593798743...",42.521176,-71.068130,02148,Massachusetts,MBTA Commuter Rail Train #214,v-1580099868,NaN
2,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,4fc4b26ee4b02051856d9e23,NaN,US,NaN,United States,7:30AM Inbound,5278,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.52199837297112...",42.521998,-71.107509,NaN,Massachusetts,MBTA Commuter Rail Train #260,v-1580099868,NaN
3,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,5b7a9b1d531593002c415517,Haverhill to North Station,US,Reading,United States,NaN,5248,"[Haverhill to North Station, Reading, MA 01867...","[{'label': 'display', 'lat': 42.52195, 'lng': ...",42.521950,-71.107140,01867,MA,Mbta Commuter Rail Train #204,v-1580099868,NaN
4,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,5343dce7498e262e20cea692,12PM Haverhill to North Station,US,Wakefield,United States,NaN,3554,"[12PM Haverhill to North Station, Wakefield, M...","[{'label': 'display', 'lat': 42.51395034790039...",42.513950,-71.086075,NaN,MA,MBTA Commuter Rail Train #220,v-1580099868,NaN


In [50]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,MBTA Commuter Rail Train #200,Metro Station,NaN,US,Reading,United States,NaN,5278,"[Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.521294, 'lng':...",42.521294,-71.107560,01867,MA,5832d1d0dfa6ff0f03038b5a
1,MBTA Commuter Rail Train #214,Train,12:41pm Haverhill to Boston,US,NaN,United States,NaN,2051,"[12:41pm Haverhill to Boston, MA 02148, United...","[{'label': 'display', 'lat': 42.52117593798743...",42.521176,-71.068130,02148,Massachusetts,50226d7be4b079960ffad394
2,MBTA Commuter Rail Train #260,Train,NaN,US,NaN,United States,7:30AM Inbound,5278,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.52199837297112...",42.521998,-71.107509,NaN,Massachusetts,4fc4b26ee4b02051856d9e23
3,Mbta Commuter Rail Train #204,Train,Haverhill to North Station,US,Reading,United States,NaN,5248,"[Haverhill to North Station, Reading, MA 01867...","[{'label': 'display', 'lat': 42.52195, 'lng': ...",42.521950,-71.107140,01867,MA,5b7a9b1d531593002c415517
4,MBTA Commuter Rail Train #220,Train,12PM Haverhill to North Station,US,Wakefield,United States,NaN,3554,"[12PM Haverhill to North Station, Wakefield, M...","[{'label': 'display', 'lat': 42.51395034790039...",42.513950,-71.086075,NaN,MA,5343dce7498e262e20cea692
5,MBTA Commuter Rail Train #2204,Train,Haverhill To Boston,US,Wakefield,United States,NaN,3677,"[Haverhill To Boston, Wakefield, MA, United St...","[{'label': 'display', 'lat': 42.49743270874023...",42.497433,-71.077248,NaN,MA,53662ff1498e5b725e6ab340
6,MBTA Commuter Rail Train #321,Train,North Station,US,Boston,United States,NaN,3672,"[North Station, Boston, MA, United States]","[{'label': 'display', 'lat': 42.49840927124023...",42.498409,-71.078270,NaN,MA,5232f98c11d2bbaf28c2445e
7,"American Training, Inc.",Community Center,107 Audubon Rd,US,Wakefield,United States,NaN,535,"[107 Audubon Rd, Wakefield, MA 01880, United S...","[{'label': 'display', 'lat': 42.520594, 'lng':...",42.520594,-71.049453,01880,MA,57a87f42498ea46a2e6547bb
8,Train To Boston,Train,NaN,US,Wilmington,United States,NaN,7783,"[Wilmington, MA, United States]","[{'label': 'display', 'lat': 42.51133368679578...",42.511334,-71.137584,NaN,MA,4e2c5c04fa76bbf847e2d324
9,Stone Zoo Cafe/Train Ride,Zoo Exhibit,149 Pond St,US,Stoneham,United States,NaN,7449,"[149 Pond St, Stoneham, MA 02180, United States]","[{'label': 'display', 'lat': 42.46297, 'lng': ...",42.462970,-71.092930,02180,MA,5aca5c476bdee60dc841ef91


In [51]:
#create a list of train stations
train = dataframe_filtered
train

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,MBTA Commuter Rail Train #200,Metro Station,NaN,US,Reading,United States,NaN,5278,"[Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.521294, 'lng':...",42.521294,-71.107560,01867,MA,5832d1d0dfa6ff0f03038b5a
1,MBTA Commuter Rail Train #214,Train,12:41pm Haverhill to Boston,US,NaN,United States,NaN,2051,"[12:41pm Haverhill to Boston, MA 02148, United...","[{'label': 'display', 'lat': 42.52117593798743...",42.521176,-71.068130,02148,Massachusetts,50226d7be4b079960ffad394
2,MBTA Commuter Rail Train #260,Train,NaN,US,NaN,United States,7:30AM Inbound,5278,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.52199837297112...",42.521998,-71.107509,NaN,Massachusetts,4fc4b26ee4b02051856d9e23
3,Mbta Commuter Rail Train #204,Train,Haverhill to North Station,US,Reading,United States,NaN,5248,"[Haverhill to North Station, Reading, MA 01867...","[{'label': 'display', 'lat': 42.52195, 'lng': ...",42.521950,-71.107140,01867,MA,5b7a9b1d531593002c415517
4,MBTA Commuter Rail Train #220,Train,12PM Haverhill to North Station,US,Wakefield,United States,NaN,3554,"[12PM Haverhill to North Station, Wakefield, M...","[{'label': 'display', 'lat': 42.51395034790039...",42.513950,-71.086075,NaN,MA,5343dce7498e262e20cea692
5,MBTA Commuter Rail Train #2204,Train,Haverhill To Boston,US,Wakefield,United States,NaN,3677,"[Haverhill To Boston, Wakefield, MA, United St...","[{'label': 'display', 'lat': 42.49743270874023...",42.497433,-71.077248,NaN,MA,53662ff1498e5b725e6ab340
6,MBTA Commuter Rail Train #321,Train,North Station,US,Boston,United States,NaN,3672,"[North Station, Boston, MA, United States]","[{'label': 'display', 'lat': 42.49840927124023...",42.498409,-71.078270,NaN,MA,5232f98c11d2bbaf28c2445e
7,"American Training, Inc.",Community Center,107 Audubon Rd,US,Wakefield,United States,NaN,535,"[107 Audubon Rd, Wakefield, MA 01880, United S...","[{'label': 'display', 'lat': 42.520594, 'lng':...",42.520594,-71.049453,01880,MA,57a87f42498ea46a2e6547bb
8,Train To Boston,Train,NaN,US,Wilmington,United States,NaN,7783,"[Wilmington, MA, United States]","[{'label': 'display', 'lat': 42.51133368679578...",42.511334,-71.137584,NaN,MA,4e2c5c04fa76bbf847e2d324
9,Stone Zoo Cafe/Train Ride,Zoo Exhibit,149 Pond St,US,Stoneham,United States,NaN,7449,"[149 Pond St, Stoneham, MA 02180, United States]","[{'label': 'display', 'lat': 42.46297, 'lng': ...",42.462970,-71.092930,02180,MA,5aca5c476bdee60dc841ef91


In [52]:
options = ['Metro Station', 'Train'] 
  
# selecting rows based on condition 
train_df = train[dataframe_filtered['categories'].isin(options)] 
  
train_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,MBTA Commuter Rail Train #200,Metro Station,NaN,US,Reading,United States,NaN,5278,"[Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.521294, 'lng':...",42.521294,-71.107560,01867,MA,5832d1d0dfa6ff0f03038b5a
1,MBTA Commuter Rail Train #214,Train,12:41pm Haverhill to Boston,US,NaN,United States,NaN,2051,"[12:41pm Haverhill to Boston, MA 02148, United...","[{'label': 'display', 'lat': 42.52117593798743...",42.521176,-71.068130,02148,Massachusetts,50226d7be4b079960ffad394
2,MBTA Commuter Rail Train #260,Train,NaN,US,NaN,United States,7:30AM Inbound,5278,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.52199837297112...",42.521998,-71.107509,NaN,Massachusetts,4fc4b26ee4b02051856d9e23
3,Mbta Commuter Rail Train #204,Train,Haverhill to North Station,US,Reading,United States,NaN,5248,"[Haverhill to North Station, Reading, MA 01867...","[{'label': 'display', 'lat': 42.52195, 'lng': ...",42.521950,-71.107140,01867,MA,5b7a9b1d531593002c415517
4,MBTA Commuter Rail Train #220,Train,12PM Haverhill to North Station,US,Wakefield,United States,NaN,3554,"[12PM Haverhill to North Station, Wakefield, M...","[{'label': 'display', 'lat': 42.51395034790039...",42.513950,-71.086075,NaN,MA,5343dce7498e262e20cea692
5,MBTA Commuter Rail Train #2204,Train,Haverhill To Boston,US,Wakefield,United States,NaN,3677,"[Haverhill To Boston, Wakefield, MA, United St...","[{'label': 'display', 'lat': 42.49743270874023...",42.497433,-71.077248,NaN,MA,53662ff1498e5b725e6ab340
6,MBTA Commuter Rail Train #321,Train,North Station,US,Boston,United States,NaN,3672,"[North Station, Boston, MA, United States]","[{'label': 'display', 'lat': 42.49840927124023...",42.498409,-71.078270,NaN,MA,5232f98c11d2bbaf28c2445e
8,Train To Boston,Train,NaN,US,Wilmington,United States,NaN,7783,"[Wilmington, MA, United States]","[{'label': 'display', 'lat': 42.51133368679578...",42.511334,-71.137584,NaN,MA,4e2c5c04fa76bbf847e2d324
11,Mbta Commuter Rail Train #288,Train,8am Reading to North Station,US,Reading,United States,NaN,5263,"[8am Reading to North Station, Reading, MA 018...","[{'label': 'display', 'lat': 42.52161, 'lng': ...",42.521610,-71.107350,01867,MA,5be03ca5fd16bb002c000593
12,Train To Salem,Train,NaN,US,NaN,United States,NaN,9721,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.45290592062626...",42.452906,-70.965892,NaN,Massachusetts,4eaad65bcc217eb08e7f0219


In [53]:
train_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(train_map)


# add train stations to the map as green circle markers
for lat, lng, label in zip(train_df.lat, train_df.lng, train_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
        ).add_to(train_map)

# display map
train_map

In [54]:
search_query = 'Hardware'
radius = 10000
print(search_query + ' .... OK!')

Hardware .... OK!


In [55]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=Hardware&radius=10000&limit=1000'

In [56]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e6951618f43001b73fb7b'},
 'response': {'venues': [{'id': '4d3ee789978476ebabd857fc',
    'name': 'L R Hart Hardware Inc',
    'location': {'address': '442 Main St',
     'lat': 42.502301,
     'lng': -71.069062,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.502301,
       'lng': -71.069062}],
     'distance': 2815,
     'postalCode': '01880',
     'cc': 'US',
     'city': 'Wakefield',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['442 Main St',
      'Wakefield, MA 01880',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d112951735',
      'name': 'Hardware Store',
      'pluralName': 'Hardware Stores',
      'shortName': 'Hardware',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/hardware_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580099906',
    'hasPerk': False},
   {'id': '5991dd7f9411f22af38f2a80',
    'name': 'Danvers Ha

In [57]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,4d3ee789978476ebabd857fc,442 Main St,US,Wakefield,United States,NaN,2815,"[442 Main St, Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.502301, 'lng':...",42.502301,-71.069062,01880,MA,L R Hart Hardware Inc,v-1580099906,NaN
1,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,5991dd7f9411f22af38f2a80,61-65 Maple Street,US,Danvers,United States,NaN,10278,"[61-65 Maple Street, Danvers, MA 01923, United...","[{'label': 'display', 'lat': 42.56740900000000...",42.567409,-70.936581,01923,MA,Danvers Hardware Store,v-1580099906,448970151
2,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,4e7cbc71b0fbfe3c16478f0d,NaN,US,NaN,United States,NaN,10123,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.522708, 'lng':...",42.522708,-70.920010,NaN,Massachusetts,Furtado's Hardware Store,v-1580099906,NaN
3,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,4d3ee7b0978476eb4f1158fc,65 Winn St,US,Woburn,United States,NaN,9997,"[65 Winn St, Woburn, MA 01801, United States]","[{'label': 'display', 'lat': 42.481974, 'lng':...",42.481974,-71.154274,01801,MA,Woburn Paint & Hardware,v-1580099906,NaN
4,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,4dd7ffaab0fb8af380948ac9,NaN,US,Peabody,United States,NaN,4803,"[Peabody, MA, United States]","[{'label': 'display', 'lat': 42.549768, 'lng':...",42.549768,-71.002249,NaN,MA,Don's Hardware,v-1580099906,NaN


In [58]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,L R Hart Hardware Inc,Hardware Store,442 Main St,US,Wakefield,United States,NaN,2815,"[442 Main St, Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.502301, 'lng':...",42.502301,-71.069062,01880,MA,4d3ee789978476ebabd857fc
1,Danvers Hardware Store,Hardware Store,61-65 Maple Street,US,Danvers,United States,NaN,10278,"[61-65 Maple Street, Danvers, MA 01923, United...","[{'label': 'display', 'lat': 42.56740900000000...",42.567409,-70.936581,01923,MA,5991dd7f9411f22af38f2a80
2,Furtado's Hardware Store,Hardware Store,NaN,US,NaN,United States,NaN,10123,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.522708, 'lng':...",42.522708,-70.920010,NaN,Massachusetts,4e7cbc71b0fbfe3c16478f0d
3,Woburn Paint & Hardware,Hardware Store,65 Winn St,US,Woburn,United States,NaN,9997,"[65 Winn St, Woburn, MA 01801, United States]","[{'label': 'display', 'lat': 42.481974, 'lng':...",42.481974,-71.154274,01801,MA,4d3ee7b0978476eb4f1158fc
4,Don's Hardware,Hardware Store,NaN,US,Peabody,United States,NaN,4803,"[Peabody, MA, United States]","[{'label': 'display', 'lat': 42.549768, 'lng':...",42.549768,-71.002249,NaN,MA,4dd7ffaab0fb8af380948ac9
5,Ace Hardware,Hardware Store,164 Chestnut St,US,North Reading,United States,NaN,6512,"[164 Chestnut St, North Reading, MA 01864, Uni...","[{'label': 'display', 'lat': 42.55466528461856...",42.554665,-71.106239,01864,MA,4d3ee762978476eb369d57fc
6,Round's Hardware,Hardware Store,290 Main St,US,Stoneham,United States,Montvale Ave.,6173,"[290 Main St (Montvale Ave.), Stoneham, MA 021...","[{'label': 'display', 'lat': 42.48229600000000...",42.482296,-71.099682,02180,MA,4b7d75b8f964a520cfbf2fe3
7,Rocky's Ace Hardware,Hardware Store,208 S Main St,US,Middleton,United States,NaN,7681,"[208 S Main St, Middleton, MA 01949, United St...","[{'label': 'display', 'lat': 42.580421, 'lng':...",42.580421,-71.000601,01949,MA,572e0c80498e9b1204547135
8,Mahan's Trustworthy Hardware,Automotive Shop,45 Boston St,US,Lynn,United States,NaN,8421,"[45 Boston St, Lynn, MA 01904, United States]","[{'label': 'display', 'lat': 42.477832, 'lng':...",42.477832,-70.957220,01904,MA,4d5a8b9ac88da1cd15194668
9,Sears Hardware,None,NaN,US,North Reading,United States,NaN,9035,"[North Reading, MA, United States]","[{'label': 'display', 'lat': 42.59014, 'lng': ...",42.590140,-71.096350,NaN,MA,4c0bfacfffb8c9b604146e61


In [59]:
#create a list of hardware stores
hardware = dataframe_filtered
hardware

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,L R Hart Hardware Inc,Hardware Store,442 Main St,US,Wakefield,United States,NaN,2815,"[442 Main St, Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.502301, 'lng':...",42.502301,-71.069062,01880,MA,4d3ee789978476ebabd857fc
1,Danvers Hardware Store,Hardware Store,61-65 Maple Street,US,Danvers,United States,NaN,10278,"[61-65 Maple Street, Danvers, MA 01923, United...","[{'label': 'display', 'lat': 42.56740900000000...",42.567409,-70.936581,01923,MA,5991dd7f9411f22af38f2a80
2,Furtado's Hardware Store,Hardware Store,NaN,US,NaN,United States,NaN,10123,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.522708, 'lng':...",42.522708,-70.920010,NaN,Massachusetts,4e7cbc71b0fbfe3c16478f0d
3,Woburn Paint & Hardware,Hardware Store,65 Winn St,US,Woburn,United States,NaN,9997,"[65 Winn St, Woburn, MA 01801, United States]","[{'label': 'display', 'lat': 42.481974, 'lng':...",42.481974,-71.154274,01801,MA,4d3ee7b0978476eb4f1158fc
4,Don's Hardware,Hardware Store,NaN,US,Peabody,United States,NaN,4803,"[Peabody, MA, United States]","[{'label': 'display', 'lat': 42.549768, 'lng':...",42.549768,-71.002249,NaN,MA,4dd7ffaab0fb8af380948ac9
5,Ace Hardware,Hardware Store,164 Chestnut St,US,North Reading,United States,NaN,6512,"[164 Chestnut St, North Reading, MA 01864, Uni...","[{'label': 'display', 'lat': 42.55466528461856...",42.554665,-71.106239,01864,MA,4d3ee762978476eb369d57fc
6,Round's Hardware,Hardware Store,290 Main St,US,Stoneham,United States,Montvale Ave.,6173,"[290 Main St (Montvale Ave.), Stoneham, MA 021...","[{'label': 'display', 'lat': 42.48229600000000...",42.482296,-71.099682,02180,MA,4b7d75b8f964a520cfbf2fe3
7,Rocky's Ace Hardware,Hardware Store,208 S Main St,US,Middleton,United States,NaN,7681,"[208 S Main St, Middleton, MA 01949, United St...","[{'label': 'display', 'lat': 42.580421, 'lng':...",42.580421,-71.000601,01949,MA,572e0c80498e9b1204547135
8,Mahan's Trustworthy Hardware,Automotive Shop,45 Boston St,US,Lynn,United States,NaN,8421,"[45 Boston St, Lynn, MA 01904, United States]","[{'label': 'display', 'lat': 42.477832, 'lng':...",42.477832,-70.957220,01904,MA,4d5a8b9ac88da1cd15194668
9,Sears Hardware,None,NaN,US,North Reading,United States,NaN,9035,"[North Reading, MA, United States]","[{'label': 'display', 'lat': 42.59014, 'lng': ...",42.590140,-71.096350,NaN,MA,4c0bfacfffb8c9b604146e61


In [60]:
options = ['Hardware Store'] 
  
# selecting rows based on condition 
hardware_df = hardware[dataframe_filtered['categories'].isin(options)] 
  
hardware_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,L R Hart Hardware Inc,Hardware Store,442 Main St,US,Wakefield,United States,NaN,2815,"[442 Main St, Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.502301, 'lng':...",42.502301,-71.069062,01880,MA,4d3ee789978476ebabd857fc
1,Danvers Hardware Store,Hardware Store,61-65 Maple Street,US,Danvers,United States,NaN,10278,"[61-65 Maple Street, Danvers, MA 01923, United...","[{'label': 'display', 'lat': 42.56740900000000...",42.567409,-70.936581,01923,MA,5991dd7f9411f22af38f2a80
2,Furtado's Hardware Store,Hardware Store,NaN,US,NaN,United States,NaN,10123,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.522708, 'lng':...",42.522708,-70.920010,NaN,Massachusetts,4e7cbc71b0fbfe3c16478f0d
3,Woburn Paint & Hardware,Hardware Store,65 Winn St,US,Woburn,United States,NaN,9997,"[65 Winn St, Woburn, MA 01801, United States]","[{'label': 'display', 'lat': 42.481974, 'lng':...",42.481974,-71.154274,01801,MA,4d3ee7b0978476eb4f1158fc
4,Don's Hardware,Hardware Store,NaN,US,Peabody,United States,NaN,4803,"[Peabody, MA, United States]","[{'label': 'display', 'lat': 42.549768, 'lng':...",42.549768,-71.002249,NaN,MA,4dd7ffaab0fb8af380948ac9
5,Ace Hardware,Hardware Store,164 Chestnut St,US,North Reading,United States,NaN,6512,"[164 Chestnut St, North Reading, MA 01864, Uni...","[{'label': 'display', 'lat': 42.55466528461856...",42.554665,-71.106239,01864,MA,4d3ee762978476eb369d57fc
6,Round's Hardware,Hardware Store,290 Main St,US,Stoneham,United States,Montvale Ave.,6173,"[290 Main St (Montvale Ave.), Stoneham, MA 021...","[{'label': 'display', 'lat': 42.48229600000000...",42.482296,-71.099682,02180,MA,4b7d75b8f964a520cfbf2fe3
7,Rocky's Ace Hardware,Hardware Store,208 S Main St,US,Middleton,United States,NaN,7681,"[208 S Main St, Middleton, MA 01949, United St...","[{'label': 'display', 'lat': 42.580421, 'lng':...",42.580421,-71.000601,01949,MA,572e0c80498e9b1204547135
10,North Reading Hardware,Hardware Store,NaN,US,North Reading,United States,NaN,9760,"[North Reading, MA 01864, United States]","[{'label': 'display', 'lat': 42.58821517, 'lng...",42.588215,-71.116377,01864,MA,4c2df38ee307d13aa99f0eda
11,Beden Hardware,Hardware Store,NaN,US,Lynn,United States,NaN,10083,"[Lynn, MA 01901, United States]","[{'label': 'display', 'lat': 42.46304321289062...",42.463043,-70.946701,01901,MA,50aa6d18e4b0b2e8c0fd4cba


In [61]:
hardware_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(hardware_map)


# add hardware stores to the map as green circle markers
for lat, lng, label in zip(hardware_df.lat, hardware_df.lng, hardware_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='orange',
        fill_color='orange',
        fill_opacity=0.6
        ).add_to(hardware_map)

# display map
hardware_map

In [62]:
search_query = 'Hospital'
radius = 10000
print(search_query + ' .... OK!')

Hospital .... OK!


In [63]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=Hospital&radius=10000&limit=1000'

In [64]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e6909e826ac001bdb1b49'},
 'response': {'venues': [{'id': '4b286142f964a520f19224e3',
    'name': 'Melrose - Wakefield Hospital',
    'location': {'address': '585 Lebanon St',
     'lat': 42.4598114,
     'lng': -71.0611428,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.4598114,
       'lng': -71.0611428}],
     'distance': 6749,
     'postalCode': '02176',
     'cc': 'US',
     'city': 'Melrose',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['585 Lebanon St',
      'Melrose, MA 02176',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580099956',
    'hasPerk': False},
   {'id': '4ba4e9c7f964a5204fc238e3',
    'name': 'VCA Wak

In [65]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4b286142f964a520f19224e3,585 Lebanon St,US,Melrose,United States,NaN,6749,"[585 Lebanon St, Melrose, MA 02176, United Sta...","[{'label': 'display', 'lat': 42.4598114, 'lng'...",42.459811,-71.061143,02176,MA,Melrose - Wakefield Hospital,v-1580099956,NaN
1,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",False,4ba4e9c7f964a5204fc238e3,19 Main St,US,Wakefield,United States,NaN,2987,"[19 Main St, Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.523883, 'lng':...",42.523883,-71.079107,01880,MA,VCA Wakefield Animal Hospital,v-1580099956,NaN
2,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4bad0fdbf964a520a9293be3,500 Lynnfield St,US,Lynn,United States,NaN,5560,"[500 Lynnfield St, Lynn, MA 01904, United States]","[{'label': 'display', 'lat': 42.50177119735922...",42.501771,-70.979695,01904,MA,Union Hospital,v-1580099956,NaN
3,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",False,4cbb0b29035d236a01c0dd4e,1 Bay State Rd,US,Lynnfield,United States,NaN,2110,"[1 Bay State Rd, Lynnfield, MA 01940, United S...","[{'label': 'display', 'lat': 42.5270308, 'lng'...",42.527031,-71.066599,01940,MA,Animal Hospital of Lynnfield,v-1580099956,38956151
4,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4c2de42de760c9b66a8f4549,480 Maple St,US,Danvers,United States,"Rt 62, Rt 1",9331,"[480 Maple St (Rt 62, Rt 1), Danvers, MA 01923...","[{'label': 'display', 'lat': 42.58433775942419...",42.584338,-70.972006,01923,MA,Beverly Hospital At Danvers,v-1580099956,NaN


In [66]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Melrose - Wakefield Hospital,Hospital,585 Lebanon St,US,Melrose,United States,NaN,6749,"[585 Lebanon St, Melrose, MA 02176, United Sta...","[{'label': 'display', 'lat': 42.4598114, 'lng'...",42.459811,-71.061143,02176,MA,4b286142f964a520f19224e3
1,VCA Wakefield Animal Hospital,Veterinarian,19 Main St,US,Wakefield,United States,NaN,2987,"[19 Main St, Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.523883, 'lng':...",42.523883,-71.079107,01880,MA,4ba4e9c7f964a5204fc238e3
2,Union Hospital,Hospital,500 Lynnfield St,US,Lynn,United States,NaN,5560,"[500 Lynnfield St, Lynn, MA 01904, United States]","[{'label': 'display', 'lat': 42.50177119735922...",42.501771,-70.979695,01904,MA,4bad0fdbf964a520a9293be3
3,Animal Hospital of Lynnfield,Veterinarian,1 Bay State Rd,US,Lynnfield,United States,NaN,2110,"[1 Bay State Rd, Lynnfield, MA 01940, United S...","[{'label': 'display', 'lat': 42.5270308, 'lng'...",42.527031,-71.066599,01940,MA,4cbb0b29035d236a01c0dd4e
4,Beverly Hospital At Danvers,Hospital,480 Maple St,US,Danvers,United States,"Rt 62, Rt 1",9331,"[480 Maple St (Rt 62, Rt 1), Danvers, MA 01923...","[{'label': 'display', 'lat': 42.58433775942419...",42.584338,-70.972006,01923,MA,4c2de42de760c9b66a8f4549
5,Pine Banks Animal Hospital & Kennels,Pet Service,1130 Main St,US,Malden,United States,NaN,8995,"[1130 Main St, Malden, MA 02148, United States]","[{'label': 'display', 'lat': 42.440271, 'lng':...",42.440271,-71.067972,02148,MA,4bdc2f1bc79cc928c27685e9
6,MelroseWakefield Hospital Emergency Department,Emergency Room,585 Lebanon St,US,Melrose,United States,NaN,6739,"[585 Lebanon St, Melrose, MA 02176, United Sta...","[{'label': 'display', 'lat': 42.4598614, 'lng'...",42.459861,-71.060883,02176,MA,5a9835feda5e56194673e4d7
7,Melrose-Wakefield Hospital ICU,Hospital Ward,NaN,US,Melrose,United States,NaN,6750,"[Melrose, MA, United States]","[{'label': 'display', 'lat': 42.459848, 'lng':...",42.459848,-71.061394,NaN,MA,571cfe98498e2192c44fb792
8,Division of Non-Invasive Pediatric Cardiac Ima...,Doctor's Office,10 Centennial Dr,US,Peabody,United States,NaN,7421,"[10 Centennial Dr, Peabody, MA 01960, United S...","[{'label': 'display', 'lat': 42.5262775, 'lng'...",42.526277,-70.953380,01960,MA,5d570f836a99f8000873e3ce
9,Division of Pulmonary Medicine at at Peabody |...,Medical Center,10 Centennial Dr,US,Peabody,United States,NaN,7421,"[10 Centennial Dr, Peabody, MA 01960, United S...","[{'label': 'display', 'lat': 42.5262775, 'lng'...",42.526277,-70.953380,01960,MA,5d570defc2a7a800084c590a


In [67]:
#create a list of hardware stores
hospital = dataframe_filtered
hospital

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Melrose - Wakefield Hospital,Hospital,585 Lebanon St,US,Melrose,United States,NaN,6749,"[585 Lebanon St, Melrose, MA 02176, United Sta...","[{'label': 'display', 'lat': 42.4598114, 'lng'...",42.459811,-71.061143,02176,MA,4b286142f964a520f19224e3
1,VCA Wakefield Animal Hospital,Veterinarian,19 Main St,US,Wakefield,United States,NaN,2987,"[19 Main St, Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.523883, 'lng':...",42.523883,-71.079107,01880,MA,4ba4e9c7f964a5204fc238e3
2,Union Hospital,Hospital,500 Lynnfield St,US,Lynn,United States,NaN,5560,"[500 Lynnfield St, Lynn, MA 01904, United States]","[{'label': 'display', 'lat': 42.50177119735922...",42.501771,-70.979695,01904,MA,4bad0fdbf964a520a9293be3
3,Animal Hospital of Lynnfield,Veterinarian,1 Bay State Rd,US,Lynnfield,United States,NaN,2110,"[1 Bay State Rd, Lynnfield, MA 01940, United S...","[{'label': 'display', 'lat': 42.5270308, 'lng'...",42.527031,-71.066599,01940,MA,4cbb0b29035d236a01c0dd4e
4,Beverly Hospital At Danvers,Hospital,480 Maple St,US,Danvers,United States,"Rt 62, Rt 1",9331,"[480 Maple St (Rt 62, Rt 1), Danvers, MA 01923...","[{'label': 'display', 'lat': 42.58433775942419...",42.584338,-70.972006,01923,MA,4c2de42de760c9b66a8f4549
5,Pine Banks Animal Hospital & Kennels,Pet Service,1130 Main St,US,Malden,United States,NaN,8995,"[1130 Main St, Malden, MA 02148, United States]","[{'label': 'display', 'lat': 42.440271, 'lng':...",42.440271,-71.067972,02148,MA,4bdc2f1bc79cc928c27685e9
6,MelroseWakefield Hospital Emergency Department,Emergency Room,585 Lebanon St,US,Melrose,United States,NaN,6739,"[585 Lebanon St, Melrose, MA 02176, United Sta...","[{'label': 'display', 'lat': 42.4598614, 'lng'...",42.459861,-71.060883,02176,MA,5a9835feda5e56194673e4d7
7,Melrose-Wakefield Hospital ICU,Hospital Ward,NaN,US,Melrose,United States,NaN,6750,"[Melrose, MA, United States]","[{'label': 'display', 'lat': 42.459848, 'lng':...",42.459848,-71.061394,NaN,MA,571cfe98498e2192c44fb792
8,Division of Non-Invasive Pediatric Cardiac Ima...,Doctor's Office,10 Centennial Dr,US,Peabody,United States,NaN,7421,"[10 Centennial Dr, Peabody, MA 01960, United S...","[{'label': 'display', 'lat': 42.5262775, 'lng'...",42.526277,-70.953380,01960,MA,5d570f836a99f8000873e3ce
9,Division of Pulmonary Medicine at at Peabody |...,Medical Center,10 Centennial Dr,US,Peabody,United States,NaN,7421,"[10 Centennial Dr, Peabody, MA 01960, United S...","[{'label': 'display', 'lat': 42.5262775, 'lng'...",42.526277,-70.953380,01960,MA,5d570defc2a7a800084c590a


In [68]:
options = ['Hospital','Emergency Room'] 
  
# selecting rows based on condition 
hospital_df = hospital[dataframe_filtered['categories'].isin(options)] 
  
hospital_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Melrose - Wakefield Hospital,Hospital,585 Lebanon St,US,Melrose,United States,NaN,6749,"[585 Lebanon St, Melrose, MA 02176, United Sta...","[{'label': 'display', 'lat': 42.4598114, 'lng'...",42.459811,-71.061143,02176,MA,4b286142f964a520f19224e3
2,Union Hospital,Hospital,500 Lynnfield St,US,Lynn,United States,NaN,5560,"[500 Lynnfield St, Lynn, MA 01904, United States]","[{'label': 'display', 'lat': 42.50177119735922...",42.501771,-70.979695,01904,MA,4bad0fdbf964a520a9293be3
4,Beverly Hospital At Danvers,Hospital,480 Maple St,US,Danvers,United States,"Rt 62, Rt 1",9331,"[480 Maple St (Rt 62, Rt 1), Danvers, MA 01923...","[{'label': 'display', 'lat': 42.58433775942419...",42.584338,-70.972006,01923,MA,4c2de42de760c9b66a8f4549
6,MelroseWakefield Hospital Emergency Department,Emergency Room,585 Lebanon St,US,Melrose,United States,NaN,6739,"[585 Lebanon St, Melrose, MA 02176, United Sta...","[{'label': 'display', 'lat': 42.4598614, 'lng'...",42.459861,-71.060883,02176,MA,5a9835feda5e56194673e4d7
14,The Melrose-Wakefield Hospital ER,Emergency Room,585 Lebanon St,US,Melrose,United States,NaN,6749,"[585 Lebanon St, Melrose, MA 02176, United Sta...","[{'label': 'display', 'lat': 42.45974489542358...",42.459745,-71.060716,02176,MA,4cf188787e0da1cd81399297
16,Winchester Hospital,Hospital,41 Highland Ave,US,Winchester,United States,NaN,8795,"[41 Highland Ave, Winchester, MA 01890, United...","[{'label': 'display', 'lat': 42.4659564, 'lng'...",42.465956,-71.122713,01890,MA,4b59dfb8f964a520809d28e3
19,Union Hospital Emergency Room,Emergency Room,NaN,US,Lynn,United States,NaN,5563,"[Lynn, MA 01904, United States]","[{'label': 'display', 'lat': 42.501324, 'lng':...",42.501324,-70.979885,01904,MA,59e8fda4d0a14965db76c434
24,The Winchester Hospital - Emergency,Emergency Room,NaN,US,Winchester,United States,NaN,8831,"[Winchester, MA, United States]","[{'label': 'display', 'lat': 42.465758, 'lng':...",42.465758,-71.123060,NaN,MA,52c88073498ea9238105a27f
25,Danvers State Mental Hospital,Hospital,50 Hathorne Green,US,Danvers,United States,NaN,9024,"[50 Hathorne Green, Danvers, MA 01923, United ...","[{'label': 'display', 'lat': 42.58144378677905...",42.581444,-70.973119,01923,MA,4b9b1485f964a52030f135e3
28,Melrose-Wakefield Hospital OR,Hospital,NaN,US,Melrose,United States,NaN,6678,"[Melrose, MA, United States]","[{'label': 'display', 'lat': 42.46041410192647...",42.460414,-71.060835,NaN,MA,51d31c7f498ea5edb70f7ecc


In [69]:
hospital_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(hospital_map)


# add hardware stores to the map as red/white circle markers
for lat, lng, label in zip(hospital_df.lat, hospital_df.lng, hospital_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='red',
        fill_color='white',
        fill_opacity=0.6
        ).add_to(hospital_map)

# display map
hospital_map

In [70]:
search_query = 'garden'
radius = 10000
print(search_query + ' .... OK!')

garden .... OK!


In [71]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=garden&radius=10000&limit=1000'

In [72]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e69fbdf2774001b16f9ce'},
 'response': {'venues': [{'id': '4b3800a4f964a5201e4a25e3',
    'name': 'Sichuan Garden II',
    'location': {'address': '2 Alfred St',
     'lat': 42.50177320763172,
     'lng': -71.15788900282574,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.50177320763172,
       'lng': -71.15788900282574}],
     'distance': 9596,
     'postalCode': '01801',
     'cc': 'US',
     'city': 'Woburn',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['2 Alfred St', 'Woburn, MA 01801', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'shortName': 'Chinese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580100009',
    'hasPerk': False},
   {'id': '4bf702ad13aed13a51a1eaf7',
    'name'

In [73]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b3800a4f964a5201e4a25e3,2 Alfred St,US,Woburn,United States,NaN,9596,"[2 Alfred St, Woburn, MA 01801, United States]","[{'label': 'display', 'lat': 42.50177320763172...",42.501773,-71.157889,01801,MA,Sichuan Garden II,v-1580100009,NaN
1,"[{'id': '4bf58dd8d48988d11b951735', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bf702ad13aed13a51a1eaf7,20 Archmeadow Dr,US,Danvers,United States,NaN,6696,"[20 Archmeadow Dr, Danvers, MA 01923, United S...","[{'label': 'display', 'lat': 42.55871376990496...",42.558714,-70.981982,01923,MA,Ann & Hope Garden Outlet,v-1580100009,NaN
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,4dcae644c65bccd8673a8735,153 Andover St,US,Danvers,United States,NaN,7486,"[153 Andover St, Danvers, MA 01923, United Sta...","[{'label': 'display', 'lat': 42.55770709687849...",42.557707,-70.968657,01923,MA,Olive Garden,v-1580100009,NaN
3,"[{'id': '4eb1c0253b7b52c0e1adc2e9', 'name': 'G...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f41388bc2ee912d134d84bc,190 Lincoln Ave,US,Saugus,United States,NaN,8599,"[190 Lincoln Ave, Saugus, MA 01906, United Sta...","[{'label': 'display', 'lat': 42.451499, 'lng':...",42.451499,-70.992359,01906,MA,Little Brook Farm & Garden,v-1580100009,NaN
4,"[{'id': '4f04aa0c2fb6e1c99f3db0b8', 'name': 'N...",NaN,NaN,NaN,NaN,NaN,NaN,False,4d7690ded6688cfaf6571f15,67-91 Main St,US,Stoneham,United States,NaN,5400,"[67-91 Main St, Stoneham, MA 02180, United Sta...","[{'label': 'display', 'lat': 42.496125, 'lng':...",42.496125,-71.101338,02180,MA,Garden Nails & Spa,v-1580100009,NaN


In [74]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Sichuan Garden II,Chinese Restaurant,2 Alfred St,US,Woburn,United States,NaN,9596,"[2 Alfred St, Woburn, MA 01801, United States]","[{'label': 'display', 'lat': 42.50177320763172...",42.501773,-71.157889,01801,MA,4b3800a4f964a5201e4a25e3
1,Ann & Hope Garden Outlet,Flower Shop,20 Archmeadow Dr,US,Danvers,United States,NaN,6696,"[20 Archmeadow Dr, Danvers, MA 01923, United S...","[{'label': 'display', 'lat': 42.55871376990496...",42.558714,-70.981982,01923,MA,4bf702ad13aed13a51a1eaf7
2,Olive Garden,Italian Restaurant,153 Andover St,US,Danvers,United States,NaN,7486,"[153 Andover St, Danvers, MA 01923, United Sta...","[{'label': 'display', 'lat': 42.55770709687849...",42.557707,-70.968657,01923,MA,4dcae644c65bccd8673a8735
3,Little Brook Farm & Garden,Garden Center,190 Lincoln Ave,US,Saugus,United States,NaN,8599,"[190 Lincoln Ave, Saugus, MA 01906, United Sta...","[{'label': 'display', 'lat': 42.451499, 'lng':...",42.451499,-70.992359,01906,MA,4f41388bc2ee912d134d84bc
4,Garden Nails & Spa,Nail Salon,67-91 Main St,US,Stoneham,United States,NaN,5400,"[67-91 Main St, Stoneham, MA 02180, United Sta...","[{'label': 'display', 'lat': 42.496125, 'lng':...",42.496125,-71.101338,02180,MA,4d7690ded6688cfaf6571f15
5,Garden Nails & Spa,Nail Salon,Lynn Fells Parkway,US,Saugus,United States,Broadway,4699,"[Lynn Fells Parkway (Broadway), Saugus, MA 019...","[{'label': 'display', 'lat': 42.47933958346187...",42.479340,-71.023661,01906,MA,4be9bac462c0c9283001e0d4
6,Garden Of Eden,Spa,802 Main St,US,Melrose,United States,NaN,6548,"[802 Main St, Melrose, MA 02176, United States]","[{'label': 'display', 'lat': 42.461883, 'lng':...",42.461883,-71.062401,02176,MA,5238b14411d2ebcbcbffb098
7,Imperial Jade Garden,Chinese Restaurant,39 Tuttle St,US,Wakefield,United States,NaN,3264,"[39 Tuttle St, Wakefield, MA 01880, United Sta...","[{'label': 'display', 'lat': 42.50245906412194...",42.502459,-71.076156,01880,MA,4aa17c74f964a520bd4020e3
8,Garden Of Eden,Spa,340 Main St,US,Melrose,United States,NaN,7721,"[340 Main St, Melrose, MA 02176, United States]","[{'label': 'display', 'lat': 42.45203124291982...",42.452031,-71.067794,02176,MA,50898f17e4b07020049efe1d
9,Garden World,Flower Shop,NaN,US,Saugus,United States,NaN,7905,"[Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.44992702313951...",42.449927,-71.020917,01906,MA,50c51a14e4b09b1d13934558


In [75]:
#create a list of hardware stores
garden = dataframe_filtered
garden

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Sichuan Garden II,Chinese Restaurant,2 Alfred St,US,Woburn,United States,NaN,9596,"[2 Alfred St, Woburn, MA 01801, United States]","[{'label': 'display', 'lat': 42.50177320763172...",42.501773,-71.157889,01801,MA,4b3800a4f964a5201e4a25e3
1,Ann & Hope Garden Outlet,Flower Shop,20 Archmeadow Dr,US,Danvers,United States,NaN,6696,"[20 Archmeadow Dr, Danvers, MA 01923, United S...","[{'label': 'display', 'lat': 42.55871376990496...",42.558714,-70.981982,01923,MA,4bf702ad13aed13a51a1eaf7
2,Olive Garden,Italian Restaurant,153 Andover St,US,Danvers,United States,NaN,7486,"[153 Andover St, Danvers, MA 01923, United Sta...","[{'label': 'display', 'lat': 42.55770709687849...",42.557707,-70.968657,01923,MA,4dcae644c65bccd8673a8735
3,Little Brook Farm & Garden,Garden Center,190 Lincoln Ave,US,Saugus,United States,NaN,8599,"[190 Lincoln Ave, Saugus, MA 01906, United Sta...","[{'label': 'display', 'lat': 42.451499, 'lng':...",42.451499,-70.992359,01906,MA,4f41388bc2ee912d134d84bc
4,Garden Nails & Spa,Nail Salon,67-91 Main St,US,Stoneham,United States,NaN,5400,"[67-91 Main St, Stoneham, MA 02180, United Sta...","[{'label': 'display', 'lat': 42.496125, 'lng':...",42.496125,-71.101338,02180,MA,4d7690ded6688cfaf6571f15
5,Garden Nails & Spa,Nail Salon,Lynn Fells Parkway,US,Saugus,United States,Broadway,4699,"[Lynn Fells Parkway (Broadway), Saugus, MA 019...","[{'label': 'display', 'lat': 42.47933958346187...",42.479340,-71.023661,01906,MA,4be9bac462c0c9283001e0d4
6,Garden Of Eden,Spa,802 Main St,US,Melrose,United States,NaN,6548,"[802 Main St, Melrose, MA 02176, United States]","[{'label': 'display', 'lat': 42.461883, 'lng':...",42.461883,-71.062401,02176,MA,5238b14411d2ebcbcbffb098
7,Imperial Jade Garden,Chinese Restaurant,39 Tuttle St,US,Wakefield,United States,NaN,3264,"[39 Tuttle St, Wakefield, MA 01880, United Sta...","[{'label': 'display', 'lat': 42.50245906412194...",42.502459,-71.076156,01880,MA,4aa17c74f964a520bd4020e3
8,Garden Of Eden,Spa,340 Main St,US,Melrose,United States,NaN,7721,"[340 Main St, Melrose, MA 02176, United States]","[{'label': 'display', 'lat': 42.45203124291982...",42.452031,-71.067794,02176,MA,50898f17e4b07020049efe1d
9,Garden World,Flower Shop,NaN,US,Saugus,United States,NaN,7905,"[Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.44992702313951...",42.449927,-71.020917,01906,MA,50c51a14e4b09b1d13934558


In [76]:
options = ['Garden'] 
  
# selecting rows based on condition 
garden_df = garden[dataframe_filtered['categories'].isin(options)] 
  
garden_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
26,Ford School Community Garden,Garden,49 Hollingsworth St,US,Lynn,United States,NaN,9517,"[49 Hollingsworth St, Lynn, MA 01902, United S...","[{'label': 'display', 'lat': 42.47134099999999...",42.471341,-70.947030,01902,MA,521bed0911d27bca4a7db3ee
45,Middleton Community gardens,Garden,NaN,US,NaN,United States,NaN,9137,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.58993280441566...",42.589933,-70.987218,NaN,Massachusetts,517c509ce4b028948bb6e416


In [77]:
garden_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(garden_map)


# add gardens to the map as green circle markers
for lat, lng, label in zip(garden_df.lat, garden_df.lng, garden_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='green',
        fill_color='white',
        fill_opacity=0.6
        ).add_to(garden_map)

# display map
garden_map

In [88]:
combined_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centered around the new office


# add New Office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(combined_map)

# add gyms to the map as purple circle markers
for lat, lng, label in zip(gym_df.lat, gym_df.lng, gym_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='purple',
        fill_color='purple',
        fill_opacity=0.6
        ).add_to(combined_map)

# add the stores as blue circle markers
for lat, lng, label in zip(venues1.lat, venues1.lng, venues1.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(combined_map)

# add popular spots to the map as blue circle markers
for lat, lng, label in zip(shopping.lat, shopping.lng, shopping.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(combined_map)

# add malls to the map as black circle markers
for lat, lng, label in zip(malls_df.lat, malls_df.lng, malls_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='black',
        fill_color='black',
        fill_opacity=0.6
        ).add_to(combined_map)


# add train stations to the map as green circle markers
for lat, lng, label in zip(train_df.lat, train_df.lng, train_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
        ).add_to(combined_map)
    
# add hardware stores to the map as orange circle markers
for lat, lng, label in zip(hardware_df.lat, hardware_df.lng, hardware_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='orange',
        fill_color='orange',
        fill_opacity=0.6
        ).add_to(combined_map)    

# add hospitals to the map as red/white circle markers
for lat, lng, label in zip(hospital_df.lat, hospital_df.lng, hospital_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='red',
        fill_color='white',
        fill_opacity=0.6
        ).add_to(hospital_map)
    
    
# display map
combined_map